In [ ]:
# importing libraries
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

SAVE_PATH = 'D:/Data/workspace/Dissertation/graphs/'

In [ ]:
accuracy_and_parameters_list = [
	(0.985, 62158),
	(0.985, 31079),
	(0.985-0.01, 31079),
	(0.98, 62158),
	(0.985, 31000),
	(0.98, 62000),
	(0.995, 62000),
	(0.99, 62000),
	(0.985, 62000),
	(0.98, 62000),
	(0.97, 62000),
	(0.95, 62000),
	(0.90, 62000),
	(0.80, 62000),
	(0.50, 62000),
	(0.20, 62000),
	(0.995, 31000),
	(0.99, 31000),
	(0.98, 31000),
	(0.97, 31000),
	(0.95, 31000),
	(0.90, 31000),
	(0.80, 31000),
	(0.50, 31000),
	(0.20, 31000),
	(0.995, 93000),
	(0.99, 93000),
	(0.98, 93000),
	(0.97, 93000),
	(0.95, 93000),
	(0.90, 93000),
	(0.80, 93000),
	(0.50, 93000),
	(0.20, 93000),
	(0.98, 100000),
	(0.98, 80000),
	(0.98, 60000),
	(0.98, 50000),
	(0.98, 40000),
	(0.98, 30000),
	(0.98, 20000),
	(0.995, 500000),
	(0.995, 250000),
	(0.995, 10000),
	(0.995, 50000),
	(0.995, 40000),
	(0.995, 30000),
	(0.995, 20000),
]

def fitness_metric_with_size_penalty(accuracy, parameters):
	return 2.5625 - (((1.0 - accuracy)/0.02) ** 2 + parameters / 31079.0)

for (accuracy, parameters) in accuracy_and_parameters_list:
    fitness = fitness_metric_with_size_penalty(accuracy, parameters)
    print(f"{accuracy=} {parameters=} {fitness=}")

In [ ]:
# Plot fitness function. The influence of "parameters" is hardly visible.
def function(x, y):
    return np.sin(np.sqrt(x ** 2 + y ** 2))

x = np.linspace(0.9, 1.0, 40)
y = np.linspace(0, 100000, 40)

X, Y = np.meshgrid(x, y)
Z = fitness_metric_with_size_penalty(X, Y)

fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection='3d')

ax.plot_surface(X, Y, Z, cmap='cool', alpha=0.8)

ax.set_title('3D Contour Plot of fitness_function(accuracy, parameters)', fontsize=14)
ax.set_xlabel('accuracy', fontsize=12)
ax.set_ylabel('parameters', fontsize=12)
ax.set_zlabel('z', fontsize=12)

plt.show()

In [ ]:
# Plot step size decay over generations curve
def plot_function(x, function, title, xlim=None, ylim=None):
	y = list(map(function, x))
	fig, ax = plt.subplots(figsize=(5, 3.5))
	ax.plot(x, y)
	# ax.set_xlim(0, 200)
	ax.set_xlim(0, xlim)
	ax.xaxis.set_major_locator(ticker.MultipleLocator(20))
	ax.set_ylim(0, ylim)
	ax.grid()
	plt.title(title)
#	plt.xlabel("Generation")
	plt.savefig(f"{SAVE_PATH}step_width_decay.svg", format="svg", dpi=1200)
	plt.show()

DECAY_RATE = 1/30
def step_sigma(generation):
	return (1/(1 + DECAY_RATE * generation))*0.5

# plot step width decay over 200 generations
x = np.arange(0, 200, 1)
plot_function(x, step_sigma, "Step size σ with decay rate = $1/30$", xlim=200)
print(step_sigma(99))
print(step_sigma(199))

In [ ]:
# Plot distribution of mutations starting at point 50 in [0, 100] interval
# at the start of the NAS run when σ = 0.5
MY= 50
SIGMA=50 

def gaussian(x, mu, sigma):
    return 1.0 / (np.sqrt(2.0 * np.pi) * sigma) * np.exp(-np.power((x - mu) / sigma, 2.0) / 2)

def my_normal_distribution(x):
	return gaussian(x, MY, SIGMA)

x = np.arange(0, 100, 1)
plot_function(x, my_normal_distribution, "Normal distribution N(50, 50)")


In [ ]:
# removed MIES integer mutation code
# phi = 1 - (step / mutation_state.nvars) / (1 + np.sqrt(1 + (step / mutation_state.nvars) ** 2))
# u1 = random.uniform(0.0, 1.0)
# u2 = random.uniform(0.0, 1.0)
# G1 = int(np.floor(np.log(1 - u1) / np.log(1 - phi)))
# G2 = int(np.floor(np.log(1 - u2) / np.log(1 - phi)))
# diff = G2 - G1

In [ ]:
# Consideration for expected normal value
from scipy.special import gamma

ABS_NORMAL_EXPECTATION_VALUE = np.sqrt(2/np.pi)

def expected_value(n):
	return np.sqrt(2) * gamma((n+1)/2) / gamma(n/2)

for n in range(3, 10):
	norm_list = []
	transformed_list = []
	for i in range(10000):
		normal_vector = np.array([random.gauss(0, 1) for k in range(0, n)])
		transformed_normal_vector = np.abs(normal_vector) - ABS_NORMAL_EXPECTATION_VALUE
		norm = np.linalg.norm(normal_vector)
		norm_list.append(norm)
		transformed_list.append(np.linalg.norm(transformed_normal_vector))
	print(f"{n=} mean={np.mean(norm_list):.5f}, expected={expected_value(n):.5f}, transformed={np.mean(transformed_normal_vector):.5f}")

In [ ]:
# Test plausibilty of excpected_value() function for gaussian distribution
for n in range(1, 10):r
	norm_list = []
	for i in range(10000):
		normal_vector = [random.gauss(0, 1) for k in range(0, n)]
		norm = np.linalg.norm(normal_vector)
		norm_list.append(norm)
	print(f"{n=} mean={np.mean(norm_list):.5f}, expected={expected_value(n):.5f}")

In [ ]:
# Estimate size of search space
# relevant number of quantizations assumed for each continuous values of the learning optimizer
real_values_combinations = 1000
# expected number of layers in search space
expected_conv_layers = 3
expected_pooling_layers = 3
expected_fc_layers = 2
conv_layer_combinations = (256-2+1) * (5-2+1) * (3-1+1)*4*2*2*2
pooling_layer_combinations = 2 * (5-2+1) * (3-1+1) * 2
fc_layer_combinations = 4 * (2048-64+1) * 2 * 2
learning_combinations = 3 * (real_values_combinations ** 3)
print(f"{conv_layer_combinations=} {pooling_layer_combinations=} {learning_combinations=}")
parameter_combinations = (conv_layer_combinations ** expected_conv_layers) * (pooling_layer_combinations ** expected_pooling_layers) * (fc_layer_combinations ** expected_fc_layers) * learning_combinations
layer_type_combinations = 2 ** (expected_conv_layers + expected_pooling_layers)
print(f"{parameter_combinations=:.3e} {layer_type_combinations=:.3e}")
total_combinations = parameter_combinations * layer_type_combinations
print(f"{total_combinations=:.3e}")

In [ ]:
# Estimate size of SMALL search space with max. 64 convolutional filters (instead of 256) and max 256 units in a fc layer (instead of 2048)
conv_layer_combinations = (64-2+1) * (5-2+1) * (3-1+1)*4*2*2*2
pooling_layer_combinations = 2 * (5-2+1) * (3-1+1) * 2
fc_layer_combinations = 4 * (256-64+1) * 2 * 2
learning_combinations = 3 * (real_values_combinations ** 3)
print(f"{conv_layer_combinations=} {pooling_layer_combinations=} {learning_combinations=}")
parameter_combinations = (conv_layer_combinations ** expected_conv_layers) * (pooling_layer_combinations ** expected_pooling_layers) * (fc_layer_combinations ** expected_fc_layers) * learning_combinations
layer_type_combinations = 2 ** (expected_conv_layers + expected_pooling_layers)
print(f"{parameter_combinations=:.3e} {layer_type_combinations=:.3e}")
total_combinations = parameter_combinations * layer_type_combinations
print(f"small space - {total_combinations=:.3e}")